# MAPD mod. B - Final Project
##  Streaming processing of cosmic rays using Drift Tubes detectors

The goal of this project is to reproduce a real-time processing of real data collected in a particle physics detector and publish the results in a
dashboard for live monitoring.

### Students:
    - Aidin Attar - 2048654
    - Ema Baci - 2050726
    - Mariam Hergnyan - 2040478

## Part 2 - Computation
In this part we take the data from Kafka producer and use Spark to process it.

In [1]:
import time
import findspark
import pyspark
import json

import numpy  as np
import pandas as pd

from kafka                import KafkaProducer
from kafka.admin          import KafkaAdminClient, NewTopic
 
from pyspark.sql           import functions as F
from pyspark.sql           import SparkSession
from pyspark.streaming     import StreamingContext
from pyspark.sql.types     import StructField, StructType, DoubleType, IntegerType
from pyspark.sql.functions import from_json, col, when, count, struct, collect_list
from pyspark               import SparkConf, SparkContext
from pyspark.sql.types     import StringType 

### Spark setup

In [2]:
#initialize all the required variables
findspark.init('/usr/local/spark')

In [6]:
#start session - specify port, application name, and configuration settings.

spark = SparkSession.builder\
        .master("spark://pd-master:7077")\
        .appName("Project_CosmicRays_Dashboard_application")\
        .config("spark.sql.execution.arrow.pyspark.enabled", "true")\
        .config("spark.sql.execution.arrow.pyspark.fallback.enabled", "false")\
        .config("spark.sql.streaming.forceDeleteTempCheckpointLocation", "true")\
        .config("spark.jars.packages","org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2")\
        .config("spark.ui.port", "4041")\
        .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

# default parallelism
#spark.conf.set("spark.sql.shuffle.partitions", spark.sparkContext.defaultParallelism)

spark

### Kafka Setup

In [7]:
KAFKA_BOOTSTRAP_SERVERS = 'pd-slave3:9092'

producer    = KafkaProducer(bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS)
kafka_admin = KafkaAdminClient(bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS)

In [8]:
kafka_admin.list_topics()

['topic_results', 'topic_stream', '__consumer_offsets']

In [9]:
# input dataframe representing the stream of input lines from kafka
# by connecting to the appropriate servers and topic

inputDF = spark.readStream\
        .format("kafka")\
        .option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP_SERVERS)\
        .option('subscribe', 'topic_stream')\
        .load()

inputDF.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [10]:
# the schema of the json data format used to create the messages

schema = StructType(
        [
                StructField("HEAD",        StringType()),
                StructField("FPGA",        StringType()),
                StructField("TDC_CHANNEL", StringType()),
                StructField("ORBIT_CNT",   StringType()),
                StructField("BX_COUNTER",  StringType()),
                StructField("TDC_MEAS",    StringType())
        ]
    )

In [11]:
# a new DF is created from the previous by using the pyspark.sql functions

jsonDF = inputDF.select(from_json(col("value").alias('value').cast("string"), schema).alias('value'))

jsonDF.printSchema()

root
 |-- value: struct (nullable = true)
 |    |-- HEAD: string (nullable = true)
 |    |-- FPGA: string (nullable = true)
 |    |-- TDC_CHANNEL: string (nullable = true)
 |    |-- ORBIT_CNT: string (nullable = true)
 |    |-- BX_COUNTER: string (nullable = true)
 |    |-- TDC_MEAS: string (nullable = true)



In [12]:
# flatten the jsonDF into a proper DataFrame

flatDF = jsonDF.select("value.HEAD", 
                       "value.FPGA", 
                       "value.TDC_CHANNEL",
                       "value.ORBIT_CNT",
                       "value.BX_COUNTER",
                       "value.TDC_MEAS")

flatDF.printSchema()

root
 |-- HEAD: string (nullable = true)
 |-- FPGA: string (nullable = true)
 |-- TDC_CHANNEL: string (nullable = true)
 |-- ORBIT_CNT: string (nullable = true)
 |-- BX_COUNTER: string (nullable = true)
 |-- TDC_MEAS: string (nullable = true)



In [13]:
# data-cleansing
df = flatDF.where(col("HEAD")==2)

# division of dataframe between chambers
df = df.withColumn('CHAMBER',
        when(( col( 'FPGA' ) == 0 ) & ( col( 'TDC_CHANNEL' ) >=  0 ) & ( col( 'TDC_CHANNEL' ) <  64  ), 0) \
       .when(( col( 'FPGA' ) == 0 ) & ( col( 'TDC_CHANNEL' ) >= 64 ) & ( col( 'TDC_CHANNEL' ) <= 127 ), 1) \
       .when(( col( 'FPGA' ) == 1 ) & ( col( 'TDC_CHANNEL' ) >=  0 ) & ( col( 'TDC_CHANNEL' ) <  64  ), 2) \
       .when(( col( 'FPGA' ) == 1 ) & ( col( 'TDC_CHANNEL' ) >= 64 ) & ( col( 'TDC_CHANNEL' ) <= 127 ), 3) \
       .when(( col( 'FPGA' ) == 1 ) &                                  ( col( 'TDC_CHANNEL' ) == 128 ), 4) \
       .otherwise(None))                                                                                   \
       .filter( col( 'CHAMBER' ).isNotNull() )

df.printSchema()

root
 |-- HEAD: string (nullable = true)
 |-- FPGA: string (nullable = true)
 |-- TDC_CHANNEL: string (nullable = true)
 |-- ORBIT_CNT: string (nullable = true)
 |-- BX_COUNTER: string (nullable = true)
 |-- TDC_MEAS: string (nullable = true)
 |-- CHAMBER: integer (nullable = true)



In [14]:
# scintillator time offset by Chamber
# TODO compute ABSOLUTETIME and DRIFTIME
time_offset_by_chamber = {
0: 95.0 - 1.1, # Ch 0
1: 95.0 + 6.4, # Ch 1
2: 95.0 + 0.5, # Ch 2
3: 95.0 - 2.6, # Ch 3
}

In [16]:
# json structure for the message to be sent as result

msg_json = {
    'time'     : .0,
    'epoch_id' :  0,
    'hits'     :  0,
    'CH0' : {'total_hits'  : 0,
             'histo_CH'    : {'bin_edges'  : [],
                              'bin_counts' : []
                             },
             'histo_ORB'   : {'bin_edges'  : [],
                              'bin_counts' : []
                             },
             'histo_SC'    : {'bin_edges'  : [],
                              'bin_counts' : []
                             }
            },
    'CH1' : {'total_hits'  : 0,
             'histo_CH'    : {'bin_edges'  : [],
                              'bin_counts' : []
                             },
             'histo_ORB'   : {'bin_edges'  : [],
                              'bin_counts' : []
                             },
             'histo_SC'    : {'bin_edges'  : [],
                              'bin_counts' : []
                             }
            },
    'CH2' : {'total_hits'  : 0,
             'histo_CH'    : {'bin_edges'  : [],
                              'bin_counts' : []
                             },
             'histo_ORB'   : {'bin_edges'  : [],
                              'bin_counts' : []
                             },
             'histo_SC'    : {'bin_edges'  : [],
                              'bin_counts' : []
                             }
            },
    'CH3' : {'total_hits'  : 0,
             'histo_CH'    : {'bin_edges'  : [],
                              'bin_counts' : []
                             },
             'histo_ORB'   : {'bin_edges'  : [],
                              'bin_counts' : []
                             },
             'histo_SC'    : {'bin_edges'  : [],
                              'bin_counts' : []
                             }
            },
}

In [17]:
def batch_proc(df, epoch_id):   

    '''
    Function for the batch processing of the data.
    The processing consists in retrieving
    the following informations:
        1. total number of processed hits, 
           post-clensing (1 value per batch)
        2. total number of processed hits,
           post-clensing, per chamber (4 values per batch)
        3. histogram of the counts of active TDC_CHANNEL,
           per chamber (4 arrays per batch)
        4. histogram of the total number of active
           TDC_CHANNEL in each ORBIT_CNT, per chamber 
           (4 arrays per batch)
        5. histogram of the counts of active TDC_CHANNEL,
           per chamber, ONLY for those orbits with at least
           one scintillator signal in it (4 arrays per batch)
        6. histogram of the DRIFTIME, per chamber 
           (4 arrays per batch) TODO
    
    Inputs:
        - df: spark dataframe with the data
        - epoch_id: batch index    
    '''

    start = time.time()
    
    msg_json['epoch_id']    = epoch_id

    #repartition the df DataFrame to 15 parts - and persist in cache to speedup calculations
    df.coalesce(15)
    df.persist()

    # total number of processed hits
    # post-cleansing
    start_1 = time.time()
    total_hits = df.filter(col('CHAMBER') != 4)\
                   .count()
    stop_1 = time.time() - start_1

    # total number of processed hits,
    # post-clensing, per chamber
    start_2 = time.time()
    df_counts = df.filter(col('CHAMBER') != 4) \
                  .groupBy(   'CHAMBER')       \
                  .count()                     \
                  .withColumnRenamed('count', 
                                     'ch_hits')
    stop_2 = time.time() - start_2


    # histogram of the counts of active
    # TDC_CHANNEL, per chamber.
    start_3 = time.time()
    df_counts_ch = df.filter(col('CHAMBER') != 4)            \
                     .groupBy('TDC_CHANNEL', 'CHAMBER')      \
                     .count()                                \
                     .groupBy('CHAMBER')                     \
                     .agg(struct(collect_list('TDC_CHANNEL') \
                                       .alias(  'bin_edges'),
                                 collect_list(      'count') \
                                       .alias(     'counts'),
                                )                            \
                          .alias('histo_CH')
                         )
    stop_3 = time.time() - start_3

    # histogram of the total number of active
    # TDC_CHANNEL in each ORBIT_CNT per chamber
    start_4 = time.time()
    df_counts_orb = df.filter(col('CHAMBER') != 4)                    \
                      .groupBy('ORBIT_CNT', 'CHAMBER')                \
                      .agg(F.countDistinct("TDC_CHANNEL"))            \
                      .groupBy('count(TDC_CHANNEL)', 'CHAMBER')       \
                      .count()                                        \
                      .groupBy('CHAMBER')                             \
                      .agg(struct(collect_list('count(TDC_CHANNEL)')  \
                                        .alias(         'bin_edges'),
                                  collect_list(             'count') \
                                        .alias(            'counts'),
                                 )                                    \
                           .alias('histo_ORB')
                          )
    stop_4 = time.time() - start_4


    # Histogram of the count of active TDC_CHANNEL,
    # per chamber, only for those orbits with
    # at least one scintillator in it
    
    # revision needed: not sure about this
    start_5 = time.time()
    list_scint = df.filter(col('CHAMBER') == 4)   \
                   .select('ORBIT_CNT')           \

    df_counts_sc = df.filter(col('CHAMBER') != 4 )                  \
                     .join(list_scint, on="ORBIT_CNT", how="inner") \
                     .groupBy(  'TDC_CHANNEL', 'CHAMBER'       )    \
                     .count()                                       \
                     .groupBy('CHAMBER')                            \
                     .agg(struct(collect_list('TDC_CHANNEL')        \
                                         .alias('bin_edges'),
                                 collect_list(      'count')        \
                                         .alias(   'counts'))       \
                          .alias('histo_SC')
                         )
    stop_5 = time.time() - start_5

    # dataframe with results joined
    start_6 = time.time()
    df_res = df_counts.join(df_counts_ch,  on='CHAMBER') \
                      .join(df_counts_orb, on='CHAMBER') \
                      .join(df_counts_sc,  on='CHAMBER') \
                      .sort('CHAMBER')                   \
                      .collect()
    stop_6 = time.time() - start_6


    start_6 = time.time()
    msg_json['hits'] = total_hits
    for i in range(len(df_res)):
        msg_json[f'CH{i}']['total_hits']               = df_res[i][  'ch_hits']
        msg_json[f'CH{i}'][  'histo_CH'][ 'bin_edges'] = df_res[i][ 'histo_CH']['bin_edges']
        msg_json[f'CH{i}'][  'histo_CH']['bin_counts'] = df_res[i][ 'histo_CH'][   'counts']
        msg_json[f'CH{i}'][ 'histo_ORB'][ 'bin_edges'] = df_res[i]['histo_ORB']['bin_edges']
        msg_json[f'CH{i}'][ 'histo_ORB']['bin_counts'] = df_res[i]['histo_ORB'][   'counts']
        msg_json[f'CH{i}'][  'histo_SC'][ 'bin_edges'] = df_res[i][ 'histo_SC']['bin_edges']
        msg_json[f'CH{i}'][  'histo_SC']['bin_counts'] = df_res[i][ 'histo_SC'][   'counts']
    stop_6 = time.time() - start_6

    end = time.time()
    
    # get the execution time
    msg_json['time'] = end - start
    
    print(f'Time 1: {stop_1}\nTime 2: {stop_2}\nTime 3: {stop_3}\nTime 4: {stop_4}\nTime 5: {stop_5}\nTime 6: {stop_6}\n', )
    
    producer.send('topic_results', json.dumps(msg_json).encode('utf-8'))
    producer.flush()

    df.unpersist()
    pass

In [18]:
df.isStreaming

True

In [19]:
# call the operations for each batch of data
while True:
    try:
        df.writeStream\
            .foreachBatch(batch_proc)\
            .trigger(processingTime='5 second')\
            .start()\
            .awaitTermination()
    except KeyboardInterrupt:
        break

Time 1: 11.879077434539795
Time 2: 0.06274032592773438
Time 3: 0.12754607200622559
Time 4: 0.1378319263458252
Time 5: 0.12026739120483398
Time 6: 4.291534423828125e-06



Time 1: 2.7541396617889404
Time 2: 0.02556443214416504
Time 3: 0.045014142990112305
Time 4: 0.07655119895935059
Time 5: 0.07497143745422363
Time 6: 0.0001308917999267578



Time 1: 1.3464953899383545
Time 2: 0.06503963470458984
Time 3: 0.19674181938171387
Time 4: 0.05465507507324219
Time 5: 0.06048226356506348
Time 6: 0.00014519691467285156



Time 1: 0.7933800220489502
Time 2: 0.03138303756713867
Time 3: 0.05147528648376465
Time 4: 0.055440664291381836
Time 5: 0.07463240623474121
Time 6: 0.00014853477478027344



Time 1: 0.48408961296081543
Time 2: 0.01795196533203125
Time 3: 0.036212921142578125
Time 4: 0.05143880844116211
Time 5: 0.06176257133483887
Time 6: 0.00013113021850585938



...

Time 1: 0.18202471733093262
Time 2: 0.010953664779663086
Time 3: 0.0274960994720459
Time 4: 0.04558300971984863
Time 5: 0.03833889961242676
Time 6: 0.0001533031463623047



Time 1: 0.18865203857421875
Time 2: 0.010695934295654297
Time 3: 0.025726318359375
Time 4: 0.03782153129577637
Time 5: 0.03840208053588867
Time 6: 0.00010824203491210938



Time 1: 0.14989519119262695
Time 2: 0.011139869689941406
Time 3: 0.02707695960998535
Time 4: 0.04084205627441406
Time 5: 0.03920412063598633
Time 6: 0.00012135505676269531



Time 1: 0.1709461212158203
Time 2: 0.011390447616577148
Time 3: 0.03180050849914551
Time 4: 0.04123067855834961
Time 5: 0.04259681701660156
Time 6: 0.00010085105895996094



ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/aidin/.local/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/aidin/.local/lib/python3.9/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/anaconda3/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt
